In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
import random
from synonyms import generate_synonym
# from torch.utils.tensorboard import SummaryWriter

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


Hello


# 1. Embedding

In [2]:
# load datasets
df = pd.read_csv('./data/hackathon_train.csv', encoding='cp949', index_col=0)

In [3]:
for i in range(10):
    print(df['Answer'].iloc[i])

<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 친구와만 지냅니다.
<중립>  다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며 요즘은 새에 관심이 조금 있습니다. 흥미가 있어도 관심도가 오래가진 않아 중립으로 하였습니다.
<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 영화관도 잘 가지 않습니다. 머리로는 안 울고 싶은데 이미 울고 있어서 당황스러울 때가 많습니다.
<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고 변수가 생길 때 세워둔 대책이 없으면 머릿속이 하얘져서 대처하기 힘들기 때문입니다. 하지만 너무 많은 변수를 생각하다가 귀찮아져서 대비책을 세우는 것을 중간에 그만둡니다.
<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대로 못합니다. 면접을 볼 때 화상 면접이었는데도 스스로 더한 압박감을 느껴 연습할 때 보다 훨씬 말하지 못했습니다.
<그렇다> 저는 파티나 행사에 간 적이 거의 없으며 간다고 가정을 하면 알고 있는 사람과 대화하는 편입니다. 이유는 모르는 사람에게 자신을 소개하는 것이 부담스럽습니다.
<아니다> 저는 여러 가지 프로젝트를 문어발식으로 조금씩 시작하여 진행이 잘 되는 프로젝트를 먼저 끝내는 방식으로 일을 합니다. 학교 다닐 때 과제가 여러 개가 있으면 하나를 하다가 집중력이 떨어지거나 막히는 일이 있을 때 다른 과제로 옮겨가는 식으로 하였습니다.
<그렇다> 매우 감상적인 편입니다. 책이나 드라마 영화 등 매체를 볼 때 무언가 다치거나 아프면 굉장히 슬퍼하고 우울해합니다.
<중립> 계획 세우는 일을 좋아하진 않지만, 여행 계획은 시간 단위로 세세하게 세웁니다. 많이 나가지 않다 보니 그 여행지에 어떤 것이 있는지 놓치고 싶지 않고 더 알차게 보내고 싶어서입니다. 다이어리는 사지만 쓰다가 안 쓰는 것을 반복하는 편입니다.
<중립> 가끔 집안 가게에서 매대를 보는데 계산 실수를 

In [4]:
from tokenization_hanbert import HanBertTokenizer
# load pretrained model
# model_name = 'kykim/bert-kor-base'
# model_name = 'monologg/kobigbird-bert-base'
# model_name = 'beomi/kcbert-base'
model_name = 'snunlp/KR-BERT-char16424'
# model_name = 'kykim/albert-kor-base'
# model_name = 'HanBert-54kN-torch'
# model_name = 'Twitter/twhin-bert-base'
# model_name = 'klue/roberta-large'

# model_name = 'klue/bert-base'
# model_name = 'klue/roberta-base'
# model_name = 'skt/kogpt2-base-v2'
def get_model(model_name, embed_dim):
    # * Model          | Tokenizer          | Pretrained weights shortcut
    # MODEL=(DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')
    # tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2")
    # model = GPT2Model.from_pretrained("skt/kogpt2-base-v2")
    # tokenizer = HanBertTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    # model = BertModel.from_pretrained(model_name)
    # use this one for albert
    # model = AlbertModel.from_pretrained(model_name)
    # model = BertModel.from_pretrained("kykim/bert-kor-base")
    # n_hl = model.config.num_hidden_layers
    n_hl = 20
    # embed_dim = model.config.embedding_size
    
    return model, tokenizer, n_hl, embed_dim

In [5]:
model, tokenizer, n_hl, embed_dim = get_model(model_name, 768)
# model_korean, tokenizer_korean, n_hl_korean, embed_dim_korean = get_model(model_name, 128)


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--snunlp--KR-BERT-char16424/snapshots/47521960ac7595c5d2ed643f7a9dab9b0efcf58d/config.json
Model config BertConfig {
  "_name_or_path": "snunlp/KR-BERT-char16424",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 16424
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--snunlp--KR-BERT-char16424/snapshots/47521960ac7595c5d2ed643f7a9dab9b0efcf58d/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.js

In [6]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(16424, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [7]:
def convert_mbti_to_label(mbti: str):
    """
    :param mbti: string. length=4
    :return:
    """
    stand = 'ISTJ'  # [0, 0, 0, 0]
    result = []
    for i in range(4):
        if stand[i] == mbti[i]:
            result.append(0)
        else:
            result.append(1)

    return result
# stand = 'ISTJ'
# label = df['MBTI'].map(convert_mbti_to_label)
# for i in range(4):
#     df['is_{}'.format(stand[i])] = label.map(lambda x: x[i])
# df

In [8]:
# # now let's see the correlation between each mbti columns and "Gender" as well as "Age"
# # let's utilize "groupby" function
# for col_name in ['Gender', 'Age']:    
#     for label in stand:
#         print('correlation between {} and {}'.format(col_name, label))
#         print(df.groupby(f'is_{label}')[f'{col_name}'].describe())
#         print('correlation value: ', df[f'is_{label}'].corr(df[f'{col_name}']))

In [9]:
# df['Age_norma'] = df['Age'].map(lambda x: (x - df['Age'].mean()) / df['Age'].std())
# df

In [10]:
# # map the gender to '남
# df['Gender_korean'] = df['Gender'].apply(lambda x: '남자' if x == 0 else '여자')
# # df['Age_enc'] = df['Age'].apply(lambda x: 0 if x == 10 else 1 if x == 20 else 2 if x == 30 else 3 if x == 40 else 4)
# df['Age_korean'] = df['Age'].apply(lambda x: '30' if x == 30 else '40' if x == 40 else '20')
# # df['Age_korean'] = df['Age'].apply(lambda x: '서른 살 (30)' if x == 30 else '마흔 살 (40)' if x == 40 else '스무 살 (20)')

# df['One_word_ans'] = df['Answer'].apply(lambda x: (x.split('>')[0])[x.index('<')+1:])
# # df['One_word_ans'] = df['One_word_ans'].astype('category').cat.codes
# df['One_word_ans'].unique()

df['Answer'] = df['Answer'].apply(lambda x: x.replace(f"<{(x.split('>')[0])[x.index('<')+1:]}>", ''))

df_question = pd.read_excel('./data/Question.xlsx', sheet_name='Sheet1', index_col=0)
df_question['Question']
df['Question'] = ''
for i, row in df.iterrows():
    df.loc[i, 'Question'] = df_question.loc[row['Q_number'], 'Question']

In [11]:
# # concatenate 'Gender_korean', 'Age_korean', 'One_word_ans' with ' ' (space)
# # df['concatenated_korean'] = '질문:' + df['Question'] + '_정답:' + df['Answer']
# df['concatenated_korean'] = df['Age_korean'] + '세의 ' + df['Gender_korean'] + '는 ' + df['Question'] + '질문에 ' + df['Answer'] + '라는 대답을 했습니다'
# # df['concatenated_korean'] = 
# #df['concatenated_korean'] = df['Answer'] ----- only ans
# # '30세의 남자는 {Q} 질문에 {A}라는 대답을 했습니다.'
# # df['Gender_korean'] + '_' + df['Age_korean'] + '_질문:' + df['Question'] + '_단답:' + df['One_word_ans'] + '_정답:' + df['Answer']

# df['concatenated_korean'] 

In [12]:
import random
# import pandas as pd
# Replace this with your DataFrame
# df = pd.read_csv('./data/hackathon_train.csv', encoding='cp949', index_col=0)


def swap_words(text, word_list):
    words = text.split()
    if len(words) > 1:
        idx = random.randrange(len(words) - 1)
        words[idx], words[idx + 1] = words[idx + 1], words[idx]
        # print(f'{words[idx]} <-> {words[idx+1]}')
    return ' '.join(words)

def insert_word(text, word_list):
    words = text.split()
    idx = random.randrange(len(words) + 1)
    new_word = random.choice(word_list)
    words.insert(idx, new_word)
    # print(f'inserted {new_word} at index {idx}')
    return ' '.join(words)

def delete_word(text, word_list):
    words = text.split()
    if len(words) > 1:
        idx = random.randrange(len(words))
        # print(f'deleted {words[idx]} at index {idx}')
        del words[idx]
        
    return ' '.join(words)

def augment_operation(df, prob, operation, row_name='Answer', word_list=None):
    new_rows = []
    for index, row in df.iterrows():
        if random.random() < prob:
            new_row = row.copy()
            new_row[row_name] = operation(row[row_name], word_list)
            new_rows.append(new_row)
    return pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)


# Apply the augmentations
def augment(df, word_list, prob_syn=0.1, prob_swap=0.1, prob_ins=0.0, prob_del=0.0, row_name='Answer'):
    
    df_augmented = df.copy()
    
    # Swap words
    df_augmented = augment_operation(df_augmented, prob_swap, swap_words, row_name)
    
    # Insert words
    df_augmented = augment_operation(df_augmented, prob_ins, insert_word, row_name, word_list)
    
    # Delete words
    df_augmented = augment_operation(df_augmented, prob_del, delete_word, row_name)
    
    # generate synonyms
    df_augmented = augment_operation(df_augmented, prob_syn, generate_synonym, row_name)
    
    print(f'original size: {df.shape}')
    # print(df)
    print(f'augmented size: {df_augmented.shape}')

    return df_augmented


In [13]:
# List of words to insert (for the insert_word function)
word_list = ['제 생각에는', '사실 그것은']
# df_augmented = augment(df, prob, word_list, swap_words=True, insert_words=True, delete_words=True)

In [14]:
# df_augmented.head(10)

In [15]:
import re
from konlpy.tag import Okt
from hanspell import spell_checker
import json
from num2words import num2words
from kss import split_sentences

def sentence_segmentation(text):
    sentences = split_sentences(text)
    return sentences

def pos_tagging(text):
    okt = Okt()
    tagged = okt.pos(text)
    return tagged



def preprocess_text(text, apply_ngrams=False, ngram_size=1, apply_replacements=True, apply_normalization=True, remove_stopwords=False):
    # Check and correct spelling
    # checked_spelling = spell_checker.check(text)
    # text = checked_spelling.checked

    # Remove special characters using regular expressions
    # text = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\s]", " ", text)

    # Initialize the Okt tokenizer
    okt = Okt()

    # Tokenize the text into words
    # words = okt.morphs(text, stem=False)
    words = text.split()
    # print(words)

    # if apply_ner:
    #     # Perform named entity recognition
    #     entities = okt.nouns(text)
    #     words = ["[NER]" if word in entities else word for word in words]

    if apply_replacements:
        # Replace specific words or phrases (example)
        replacements = {"아니다": "아닙니다", "그렇다": "그렇습니다", "이다": "입니다"}
        words = [replacements.get(word, word) for word in words]

    if apply_normalization:
        # Perform text normalization 
        words = [num2words(word) if word.isdigit() else word for word in words]
        words = [okt.normalize(word) for word in words]
        # words = [normalization_map.get(word, word) for word in words]

    # Remove stopwords
    if remove_stopwords:
        stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '다']
        words = [word for word in words if word not in stopwords]

    # Remove short and long words
    # words = [word for word in words if 3 <= len(word) <= 30]

    if apply_ngrams:
        # Generate N-grams
        ngrams = zip(*[words[i:] for i in range(ngram_size)])
        ngrams = [" ".join(ngram) for ngram in ngrams]
        words.extend(ngrams)

    return " ".join(words)




In [16]:
# example to see the result of preprocessing
ans = df['Answer'].iloc[5]
print(ans)

print(preprocess_text(ans, apply_ngrams=True, ngram_size=2, apply_replacements=True, apply_normalization=True, remove_stopwords=False))
# print(pos_tagging(ans))

 저는 파티나 행사에 간 적이 거의 없으며 간다고 가정을 하면 알고 있는 사람과 대화하는 편입니다. 이유는 모르는 사람에게 자신을 소개하는 것이 부담스럽습니다.
저는 파티나 행사에 간 적이 거의 없으며 간다고 가정을 하면 알고 있는 사람과 대화하는 편입니다. 이유는 모르는 사람에게 자신을 소개하는 것이 부담스럽습니다. 저는 파티나 파티나 행사에 행사에 간 간 적이 적이 거의 거의 없으며 없으며 간다고 간다고 가정을 가정을 하면 하면 알고 알고 있는 있는 사람과 사람과 대화하는 대화하는 편입니다. 편입니다. 이유는 이유는 모르는 모르는 사람에게 사람에게 자신을 자신을 소개하는 소개하는 것이 것이 부담스럽습니다.


In [17]:
# df = df_augmented.copy()

In [18]:
from collections import Counter

processed_ans = preprocess_text(ans, apply_ngrams=False, ngram_size=1, apply_replacements=True, apply_normalization=True)
preprocessed_ans = processed_ans.split(' ')
# print(preprocessed_ans)
cnt = Counter(preprocessed_ans)
cnt.most_common(2)

[('저는', 1), ('파티나', 1)]

In [19]:
# now preprocess the entire 'Answer' column
df['preprocessed_ans'] = df['Answer'].apply(preprocess_text)

In [20]:
stand = 'ISTJ'
df['labels'] = df['MBTI'].map(convert_mbti_to_label)
for i in range(4):
    df['is_{}'.format(stand[i])] = df['labels'].map(lambda x: x[i])
df

,User_ID,Gender,Age,MBTI,Q_number,Answer,Question,preprocessed_ans,labels,is_I,is_S,is_T,is_J
Data_ID,,,,,,,,,,,,,
1,1,1,30,INFP,1,어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 친구와만...,주기적으로 새로운 친구를 만드나요? 경험을 비추어봤을 때 어떤지와 그러한 이유가 궁...,어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 친구와만 ...,"[0, 1, 1, 1]",0,1,1,1
2,1,1,30,INFP,2,다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며 요즘...,자유 시간 중 상당 부분을 다양한 관심사를 탐구하는 데 할애하나요? 요즘 어떤 관심...,다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며 요즘은 ...,"[0, 1, 1, 1]",0,1,1,1
3,1,1,30,INFP,3,감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 영화관도 ...,다른 사람이 울고 있는 모습을 보면 자신도 울고 싶어질 때가 많나요? 이런 상황에서...,감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 영화관도 잘...,"[0, 1, 1, 1]",0,1,1,1
4,1,1,30,INFP,4,대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고 변수가...,일이 잘못될 때를 대비해 여러 대비책을 세우는 편인가요? 이유는 무엇인가요.,대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고 변수가 ...,"[0, 1, 1, 1]",0,1,1,1
5,1,1,30,INFP,5,평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대로 못합니...,압박감이 심한 환경에서도 평정심을 유지하는 편인가요? 최근 경험을 말씀해주세요.,평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대로 못합니다...,"[0, 1, 1, 1]",0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14384,240,0,40,ISTJ,44,저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획으로 인한...,계획에 차질이 생기면 최대한 빨리 계획으로 돌아가기 위해 노력하나요? 최근에 있었던...,저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획으로 인한 ...,"[0, 0, 0, 0]",0,0,0,0
14385,240,0,40,ISTJ,45,저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적도 많기 ...,오래전의 실수를 후회할 때가 많나요? 요즘에 무엇때문에 그런가요.,저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적도 많기 때...,"[0, 0, 0, 0]",0,0,0,0
14386,240,0,40,ISTJ,46,저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이미 정해진...,인간의 존재와 삶의 이유에 대해 깊이 생각하지 않는 편인가요? 답변에 대한 이유를 ...,저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이미 정해진 ...,"[0, 0, 0, 0]",0,0,0,0


In [21]:
# split train and test dataframe
train_df_list = []
test_df_list = []
for idx in df['User_ID'].unique():
    train_df_list.append(df[df['User_ID']==idx][0:40])
    test_df_list.append(df[df['User_ID']==idx][40:])

train_df = pd.concat(train_df_list, ignore_index=True)
test_df = pd.concat(test_df_list, ignore_index=True)

In [22]:
# augment the train dataframe
prob_syn, prob_swap, prob_ins, prob_del = 0.0, 0.1, 0.1, 0.1
train_df_augmented = augment(train_df, word_list, prob_syn=prob_syn, prob_swap=prob_swap, prob_ins=prob_ins, prob_del=prob_del, row_name='preprocessed_ans')
train_df = train_df_augmented.copy()
# test_df_augmented = augment(test_df, word_list, prob_syn=prob_syn, prob_swap=prob_swap, prob_ins=prob_ins, prob_del=prob_del, row_name='preprocessed_ans')
# test_df = test_df_augmented.copy()

original size: (9600, 13)
augmented size: (12778, 13)


In [23]:
# # let's add number of words as a feature
# train_df['num_words'] = train_df['preprocessed_ans'].apply(lambda x: len(x.split()))
# test_df['num_words'] = test_df['preprocessed_ans'].apply(lambda x: len(x.split()))
# train_df['num_sentences'] = train_df['preprocessed_ans'].apply(lambda x: len(sentence_segmentation(x)))
# test_df['num_sentences'] = test_df['preprocessed_ans'].apply(lambda x: len(sentence_segmentation(x)))

# # let's compare the number of words between introverts and extroverts
# train_df['num_words_per_sentence'] = train_df['num_words'] / train_df['num_sentences']
# test_df['num_words_per_sentence'] = test_df['num_words'] / test_df['num_sentences']
# train_df['num_words_per_sentence'].groupby(train_df['is_J']).describe()
# test_df['num_words_per_sentence'].groupby(test_df['is_J']).describe()

In [24]:
# # let's see more the introverts' answers
# words_dict = {} # count the frequency of each words

# intro_df = df[df['is_S'] == 1] # change the label to see the other mbti types

# for i, row in intro_df.iterrows():
#     words = preprocess_text(row['Answer']).split()
#     for word in words:
#         if word in words_dict:
#             words_dict[word] += 1
#         else:
#             words_dict[word] = 1
# # wrap the words_dict into 'Counter' class
# cnt_words = Counter(words_dict)
# cnt_words.most_common(30) # we will decide the frequency of which words to be tracked


In [25]:
# extro_cnt_words = {}
# extro_df = df[df['is_S'] == 0]

# for i, row in extro_df.iterrows():
#     words = preprocess_text(row['Answer']).split()
#     for word in words:
#         if word in extro_cnt_words:
#             extro_cnt_words[word] += 1
#         else:
#             extro_cnt_words[word] = 1

# extro_cnt_words = Counter(extro_cnt_words)
# extro_cnt_words.most_common(30) # we will decide the frequency of which words to be tracked


In [26]:
# # find words which are more frequently used by introverts than extroverts
# common_extro_cnt_words = [word for word, _ in extro_cnt_words.most_common(30)]
# common_cnt_words = [word for word, _ in cnt_words.most_common(30)]

# only_intro_words = [word for word in common_cnt_words if word not in common_extro_cnt_words]
# only_extro_words = [word for word in common_extro_cnt_words if word not in common_cnt_words]
# # len(only_intro_words), len(only_extro_words)
# only_extro_words

In [27]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Assuming your DataFrame is named `df` and has a column named "Answer"
# # containing the preprocessed text data.

# # Initialize the TfidfVectorizer
# vectorizer = TfidfVectorizer()

# # Calculate the TF-IDF values for the "Answer" column
# tfidf_matrix = vectorizer.fit_transform(df['Answer'].apply(preprocess_text).values)

# # You can convert the TF-IDF matrix to a DataFrame if needed
# tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
# # Now, `tfidf_df` contains the calculated TF-IDF values for each word in each document.
# tfidf_df

In [28]:
# find the column name of the max value in each row
# tfdf['freq_word'] = tfidf_df.idxmax(axis=1)
# df['freq_word']
# find the number of 'Nan's in the column named 'freq_word'

# num_nans = df['freq_word']
# tfidf_df['freq_word'] = tfidf_df.idxmax(axis=1)
# df['freq_word'] = tfidf_df['freq_word']
# df['freq_word'].nunique()

In [29]:
# # set a pad token if you are using GPT2
# # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# # be careful to comment this only if you tokenize the "Answer" column 
# train_df['Answer_rep'] = train_df['Answer'].apply(lambda x: x.replace(f"<{(x.split('>')[0])[x.index('<')+1:]}>", ''))
# test_df['Answer_rep'] = test_df['Answer'].apply(lambda x: x.replace(f"<{(x.split('>')[0])[x.index('<')+1:]}>", ''))
# train_df["preprocessed_answer"] = train_df["Answer_rep"].apply(preprocess_text)
# test_df["preprocessed_answer"] = test_df["Answer_rep"].apply(preprocess_text)

train_df.shape, test_df.shape

((12778, 13), (1920, 13))

In [30]:
# if you are eliminating the one-word-short answer -> please modify Answer to Answer_rep

train_tensor = tokenizer(train_df['preprocessed_ans'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True, truncation=True)
test_tensor = tokenizer(test_df['preprocessed_ans'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True, truncation=True)
# train_tensor = tokenizer(train_df['concatenated_korean'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
# test_tensor = tokenizer(test_df['concatenated_korean'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)


In [31]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        data = {k:v[idx] for k,v in self.data.items()}
        return data

In [32]:
def build_dataloader(tokens, batch_size=32, shuffle=False):
    ds = MyMapDataset(tokens)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)
# train_ds = MyMapDataset(train_tensor)
# test_ds = MyMapDataset(test_tensor)

train_tensor_dl = build_dataloader(train_tensor, batch_size=32, shuffle=False)
test_tensor_dl = build_dataloader(test_tensor, batch_size=32, shuffle=False)
# train_tensor_korean_dl = build_dataloader(train_tensor_korean, batch_size=32, shuffle=False)
# test_tensor_korean_dl = build_dataloader(test_tensor_korean, batch_size=32, shuffle=False)

In [33]:
def forward(model, dl, device=0):
    pooled = []
    hidden = []
    model.cuda(device)
    model.eval()
    for data in dl:
        data = {k:v.cuda(device) for k,v in data.items()}
        with torch.no_grad():
            output = model(**data, output_hidden_states=True)
            # input_ids = data['input_ids']
            # output = model(**data, return_dict=True)
        p, h = output.pooler_output, output.hidden_states
        # p, h = output.last_hidden_state[:,0,:], output.last_hidden_state # only [CLS] token embedding
        # p = output.hidden_states[-1][:, -1, :]
        pooled.append(p) # pooler output
        hidden.append(h[-1][:,0,:]) # only [CLS] token embedding 
        # hidden.append(h) # all token embedding
    return torch.cat(pooled), torch.cat(hidden)

In [34]:
train_result = forward(model, train_tensor_dl, device=0)
test_result = forward(model, test_tensor_dl, device=1)
# train_result_korean = forward(model, train_tensor_korean_dl, device=0)
# test_result_korean = forward(model, test_tensor_korean_dl, device=1)

In [35]:
train_result[0].shape, test_result[0].shape

(torch.Size([12778, 768]), torch.Size([1920, 768]))

In [36]:
train_result[1].shape

torch.Size([12778, 768])

In [37]:
# torch.save(train_result, f'{model_name.replace("/", "_")}_train_preprocessed.pt')
# torch.save(test_result, f'{model_name.replace("/", "_")}_test_preprocessed.pt')

In [38]:
train_result[0].shape

torch.Size([12778, 768])

In [39]:
train_result, test_result = train_result[0], test_result[0]

In [40]:
# added_cols = ['Gender', 'Age'] + [col for col in train_df.columns if col.startswith('is')]
train_df = pd.get_dummies(data=train_df, columns=['Age'], prefix='cat')
test_df = pd.get_dummies(data=test_df, columns=['Age'], prefix='cat')
added_cols = ['Gender'] + [col for col in train_df.columns if col.startswith('cat')]
nums = 48
# for i in range(nums):
#     for col in added_cols:
#         if col.startswith('cat'):
#             if i % 6 == 0:
#                 added_cols.append(col)
#         else:
#             added_cols.append(col)

for _ in range(nums):
    added_cols += ['Gender']

for _ in range(nums//6):
    # added_cols += ['num_sentences', 'num_words_per_sentence']
    added_cols += [col for col in train_df.columns if col.startswith('cat')]


col_data = train_df[added_cols].values
test_col_data = test_df[added_cols].values
# print(col_data[0][0].shape)
col_data = col_data.astype(float)
test_col_data = test_col_data.astype(float)
print(col_data.shape)
print(train_result.shape)

col_data = torch.tensor(col_data, dtype=torch.float, device='cuda:0')
test_col_data = torch.tensor(test_col_data, dtype=torch.float, device='cuda:1')

# train_result_added = torch.cat([train_result, col_data], dim=1)
# test_result_added = torch.cat([test_result, test_col_data], dim=1)


(12778, 76)
torch.Size([12778, 768])


In [41]:
# train_result_added.shape, test_result_added.shape


In [42]:
def set_random(SEED=0):
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

class MyDataset(Dataset):
    def __init__(self, data, label, label_idx=0):
        self.data = data
        self.label = label
        self.label_idx = label_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], torch.tensor(self.label[idx][self.label_idx])
    
def convert_mbti_to_label(mbti: str):
    """
    :param mbti: string. length=4
    :return:
    """
    stand = 'ISTJ'  # [0, 0, 0, 0]
    result = []
    for i in range(4):
        if stand[i] == mbti[i]:
            result.append(0)
        else:
            result.append(1)

    return result

# MBTI = ['IE', 'SN', 'TF', 'JP']
# # stand = 'ISTJ'
# label = df['MBTI'].map(convert_mbti_to_label)
# for i in range(4):
#     df['is_{}'.format(MBTI[i])] = label.map(lambda x: x[i])
# df = df.drop(columns=['is_IE'])
    

In [43]:
from sklearn.svm import SVC
from torch.optim import Adam
# from thundersvm import SVC as ThunderSVC
import cudf
from cuml.svm import SVC
import joblib
from sklearn.metrics import roc_auc_score
gpu_id = 0
cudf.set_allocator("managed")
cudf.cuda.select_device(gpu_id)
def main(label_idx, epochs, name, test_number, col_data, test_col_data, train_label_1d, test_label_1d, train_result, test_result):
    print(f'{col_data.shape} <- col_data, {train_result.shape} <- train_result')
    
    mlp = nn.Sequential(nn.Linear(col_data.shape[1], 786),
                        #   nn.Dropout(0.35),
                          nn.ReLU(),        
                          nn.Linear(786, 3886),
                        #   nn.ReLU(),
                        #   nn.Linear(786//2, 1086),
                        #   nn.ReLU(),
                        #   nn.Linear(786, 786),
                          )

    # if chkpt_path is not None:
    #     mlp.load_state_dict(torch.load(chkpt_path))
    
    mlp = mlp.cuda(0)
    # Define the optimizer
    optimizer = Adam(mlp.parameters(), lr=1e-2)

    # input to the MLP -> col_data (they can be on cuda:0) -> output -> 768
    # then concatenate col_data and train_result[0] to give train_result_added
    # and feed the concatenated tensor to the svm
    
    # Training
    svm_model = SVC(kernel='rbf', C=1.5, gamma=0.05, probability=True)
    # if svm_path is not None:
    #     svm_model = joblib.load(svm_path)
    # test_col_data = test_col_data.cuda(0)
    
    # test_col_data = test_col_data.cuda(0)
    
    # svm_model = SVC(kernel='rbf', C=1, gamma=0.085)
    # hinge_losses = []
    # min_hinge_loss = np.inf
    max_auc = 0

    for epoch in range(epochs):
        
        # if mlp is in cuda:1 move it to cuda:0
        
        if mlp[0].weight.device != torch.device('cuda:0'):
            mlp = mlp.cuda(0)
        # Forward pass through the MLP
        # mlp_output_train = mlp(col_data) # output has shape of (9600, 768)
        mlp_output_train = mlp(col_data.to(mlp[0].weight.dtype))

        # print(f'passing {col_data} to the mlp')

        # Concatenate the output of the MLP with the additional features
        concatenated_features_train = np.hstack([mlp_output_train.detach().cpu().numpy(), train_result.detach().cpu().numpy()]) # shape of (9600, 768 + 768)
        # print(f'concatenating mlp output with the embedded features')

        # Train the SVM
        svm_model.fit(concatenated_features_train, train_label_1d) # shape of (9600, 768 + 768)

        # Compute the hinge loss
        hinge_loss = torch.nn.functional.multi_margin_loss(mlp_output_train, torch.from_numpy(train_label_1d).cuda(0), margin=1.0)

        

        # Backward pass and optimization
        optimizer.zero_grad()
        hinge_loss.backward()
        optimizer.step()


        # Testing
        with torch.no_grad():
            # first move the mlps to the cuda:1

            # mlp = mlp.cuda(1)
            test_col_data = test_col_data.cuda(0)
            test_col_data = test_col_data.to(mlp[0].weight.dtype)
            mlp_output_test = mlp(test_col_data)
            concatenated_features_test = np.hstack([mlp_output_test.detach().cpu().numpy(), test_result.detach().cpu().numpy()])
            accuracy = svm_model.score(concatenated_features_test, test_label_1d)
            # test_prob_pred = svm_model.predict_proba(concatenated_features_test)
            prob_preds = svm_model.predict_proba(concatenated_features_test)[:, 1]
            roc_auc = roc_auc_score(test_label_1d, prob_preds)
            # training_accuracy = svm_model.score(concatenated_features_train, train_label_1d)
            
            
            if roc_auc > max_auc:
                max_auc = roc_auc
                torch.save(mlp.state_dict(), f'./{name}_model_{test_number}.pth')
                # svm_model.save_to_file(f'./{name}_svm_{test_number}.pkl')
                # also save it using joblib
                joblib.dump(svm_model, f'./{name}_svm_{test_number}.pkl')
            
            # print(f'saving the models with the minimum hinge loss: {min_hinge_loss}')
            
        if (epoch+1) % 5 == 0:    
            print(f'Epoch [{epoch+1}/{epochs}], Hinge Loss: {hinge_loss.item()}, Train_acc: , Accuracy: {accuracy}, ROC AUC: {roc_auc}')
        
    # if it is the last epoch, save the predictions and add it in the next training data
        
    train_prob_preds = svm_model.predict_proba(concatenated_features_train)[:, 1]
        
    return max_auc, accuracy, train_prob_preds, prob_preds

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.8/site-packages/cudf/utils/utils.py:218: FutureWarning: The cudf.set_allocator function is deprecated and will be removed in a future release. Please use rmm.reinitialize (https://docs.rapids.ai/api/rmm/stable/api.html#rmm.reinitialize) instead. Note that `cudf.set_allocator(allocator="managed")` is equivalent to `rmm.reinitialize(managed_memory=True)`.
  warnings.warn(


In [44]:
# Train all
# suppress the warnings
import warnings
warnings.filterwarnings("ignore")
test_number = 201
epochs = 200
import csv
MBTI = ['IE', 'SN', 'TF', 'JP']
set_random(42)
best_accuracies = {}
result_dir = f'./result_{test_number}_{model_name.replace("/", "_")}'
preds = []
test_preds = []
train_label = train_df['MBTI'].map(convert_mbti_to_label)
train_label_2d = np.array(train_label.tolist())
test_label = test_df['MBTI'].map(convert_mbti_to_label)
test_label_2d = np.array(test_label.tolist())
for i in range(4):
    label_idx = i
    # convert a pandas series whose data values are list -> to a 2D numpy array
    train_label_1d = train_label_2d[:, label_idx]
    test_label_1d = test_label_2d[:, label_idx]
    # preds_tensor = torch.tensor(preds).cuda(0)
    # col_data_merged = torch.cat([col_data, test_col_data.cuda(0)], dim=0)
    # labels_merged = np.concatenate([train_label_1d, test_label_1d], axis=0)
    # # test_result = test_result.cuda(0)
    # result_merged = torch.cat([train_result, test_result.cuda(0)], dim=0)
    max_auc, accuracy, train_prob_preds, test_prob_preds = main(label_idx, epochs, MBTI[i], test_number, col_data, test_col_data, train_label_1d, test_label_1d, train_result, test_result)
    train_label_1d = torch.tensor(train_label_1d).cuda(0).reshape(-1, 1)
    test_label_1d = torch.tensor(test_label_1d).cuda(0).reshape(-1, 1)
    col_data = torch.cat([col_data, train_label_1d], dim=1)
    col_data = col_data.float()
    test_col_data = torch.cat([test_col_data.cuda(0), test_label_1d], dim=1)
    test_col_data = test_col_data.float()
    # preds.append(train_prob_preds)
    # test_preds.append(test_prob_preds)
    print('appending the predictions of', MBTI[i])
    best_accuracies[MBTI[i]] = max_auc
    print(f'Best accuracy for {MBTI[i]}: {max_auc}')

# print(best_accuracies)

torch.Size([12778, 76]) <- col_data, torch.Size([12778, 768]) <- train_result
Epoch [5/200], Hinge Loss: 0.0002588242059573531, Train_acc: , Accuracy: 0.5, ROC AUC: 0.5
Epoch [10/200], Hinge Loss: 0.0002171766827814281, Train_acc: , Accuracy: 0.5, ROC AUC: 0.5375368923611111
Epoch [15/200], Hinge Loss: 0.00021409873443190008, Train_acc: , Accuracy: 0.5, ROC AUC: 0.5005208333333333
Epoch [20/200], Hinge Loss: 0.00021322851534932852, Train_acc: , Accuracy: 0.5, ROC AUC: 0.6367420789930556
Epoch [25/200], Hinge Loss: 0.0002155217225663364, Train_acc: , Accuracy: 0.5, ROC AUC: 0.6627007378472223
Epoch [30/200], Hinge Loss: 0.0002097525430144742, Train_acc: , Accuracy: 0.5885416865348816, ROC AUC: 0.6654427083333334
Epoch [35/200], Hinge Loss: 0.00020845630206167698, Train_acc: , Accuracy: 0.6223958134651184, ROC AUC: 0.6656998697916667
Epoch [40/200], Hinge Loss: 0.00020636862609535456, Train_acc: , Accuracy: 0.6567708253860474, ROC AUC: 0.7451231553819444
Epoch [45/200], Hinge Loss: 0.000

In [45]:
best_accuracies

{'IE': 0.7683962673611111,
 'SN': 0.7907573784722223,
 'TF': 0.822121310763889,
 'JP': 0.8962489149305556}

Even though the performance of the model seem very high here, when we submitted it to kaggle it got very low accuracy.
This is because the model is overfitting the training data because it receives previous labels.